**WARNING:** This is unfinished and unpublished work - it is probably wrong - do not rely on it and do not cite it.

## EU Cage Free Model


Run Monte Carlo model in six month intervals, for each interval:

1.) Extrapolate country trends for all countries

2.) For each country, if 60% or more hens projected to be cage-free in that country, there is a 46% chance become a NATIONAL restriction in that country

3.) Each year, expect a scientific report sometime between December 2022 and 2024... when report happens, there is a 36% chance of EU PROPOSAL after 2-29 months

4.) The EU Presidency rotates through a known sequence... if (a) the EU Presidency is a country with a national law and (b) the number of countries with a national ban (`support`) is >= 15, and (c) there is a proposal, there is a 57% chance of an EU law after 12-36 months - if it hits a country that has a national law, 57% chance it becomes EU law after 12-36 months.

This assumes the EU continues to exist, has no new members altered, and is not fundamentally restructured.

In [1]:
import random

import pandas as pd
import numpy as np

from copy import copy

def print_pct(pct, digits=0):
    pct = pct * 100
    pct = np.round(pct, digits)
    if pct >= 100:
        if digits == 0:
            val = '>99.0%'
        else:
            val = '>99.'
            for d in range(digits - 1):
                val += '9'
            val += '9%'
    elif pct <= 0:
        if digits == 0:
            val = '<1.0%'
        else:
            val = '<0.'
            for d in range(digits - 1):
                val += '0'
            val += '1%'
    else:
        val = '{}%'.format(pct)
    return val

In [2]:
N_SCENARIOS = 10000

# Rotation order of the EU Presidencies
EU_PRESIDENCIES = ['Portugal', 'Slovenia', 'France', 'Czechia', 'Sweden', 'Spain', 'Belgium',
                   'Hungary', 'Poland', 'Denmark', 'Cyprus', 'Ireland', 'Lithuania', 'Greece',
                   'Italy', 'Latvia', 'Luxembourg', 'Netherlands', 'Slovakia', 'Malta',
                   'Estonia', 'Bulgaria','Austria', 'Romania', 'Finland', 'Croatia', 'Germany']
    
# France and Netherlands have partial bans, which we treat as a complete ban
COUNTRIES_W_BANS = ['Germany', 'Czechia', 'Luxembourg', 'France', 'Netherlands','Austria']
    
# % of country that would be cage-free by 2025
COUNTRY_RATES = {'Austria': 0.1,
                 'Belgium': 0.799,
                 'Bulgaria': 0.299,
                 'Croatia': 0.436,
                 'Cyprus': 0.519,
                 'Czechia': 0.366,
                 'Denmark': 0.1,
                 'Estonia': 0.286,
                 'Finland': 0.548,
                 'France': 0.572,
                 'Germany': 0.1,
                 'Greece': 0.31,
                 'Hungary': 0.317,
                 'Ireland': 0.537,
                 'Italy': 0.655,
                 'Latvia': 0.211,
                 'Lithuania': 0.171,
                 'Luxembourg': 1,
                 'Malta': 0.009,
                 'Netherlands': 0.917,
                 'Poland': 0.233,
                 'Portugal': 0.176,
                 'Romania': 0.368,
                 'Slovakia': 0.212,
                 'Slovenia': 0.716,
                 'Spain': 0.33,
                 'Sweden': 0.95}
    
# Expectation of growth in cage-free for each country over each year
COUNTRY_GROWTH = {'Austria': 0.004,
                  'Belgium': 0.037,
                  'Bulgaria': 0,
                  'Croatia': 0.02,
                  'Cyprus': 0.07,
                  'Czechia': 0.031,
                  'Denmark': 0.077,
                  'Estonia': 0.022,
                  'Finland': 0.022,
                  'France': 0.029,
                  'Germany': 0.001,
                  'Greece': 0.0056,
                  'Hungary': 0.0003,
                  'Ireland': 0.014,
                  'Italy': 0.036,
                  'Latvia': 0.006,
                  'Lithuania': 0.026,
                  'Luxembourg': 0,
                  'Malta': 0.011,
                  'Netherlands': 0.0086,
                  'Poland': 0.0128,
                  'Portugal': 0.014,
                  'Romania': 0,
                  'Slovakia': 0.001,
                  'Slovenia': 0.028,
                  'Spain': 0.0298,
                  'Sweden': 0.018}
    

In [3]:
country_rates = COUNTRY_RATES.copy()
country_growth = COUNTRY_GROWTH.copy()
years = range(2021, 2050)
support_rates = []
supporters = copy(COUNTRIES_W_BANS)

for year in years:
    print('## {} ##'.format(year))
    for country in country_rates.keys():
        country_rates[country] = np.round(country_rates[country] + country_growth[country], 3)
        if country_rates[country] > 1:
            country_rates[country] = 1
    print(country_rates)
    total_supporters = [k for k, v in country_rates.items() if v > 0.6]
    total_supporters = list(set(total_supporters) | set(supporters))
    new_supporters = list(set(total_supporters) - set(supporters))
    supporters = total_supporters
    print('--- Support: {}/15'.format(len(supporters)))
    print('--- New Supporters: {}'.format(', '.join(new_supporters)))
    support_rates.append(len(supporters))
    print('-')

## 2021 ##
{'Austria': 0.104, 'Belgium': 0.836, 'Bulgaria': 0.299, 'Croatia': 0.456, 'Cyprus': 0.589, 'Czechia': 0.397, 'Denmark': 0.177, 'Estonia': 0.308, 'Finland': 0.57, 'France': 0.601, 'Germany': 0.101, 'Greece': 0.316, 'Hungary': 0.317, 'Ireland': 0.551, 'Italy': 0.691, 'Latvia': 0.217, 'Lithuania': 0.197, 'Luxembourg': 1, 'Malta': 0.02, 'Netherlands': 0.926, 'Poland': 0.246, 'Portugal': 0.19, 'Romania': 0.368, 'Slovakia': 0.213, 'Slovenia': 0.744, 'Spain': 0.36, 'Sweden': 0.968}
--- Support: 10/15
--- New Supporters: Slovenia, Sweden, Belgium, Italy
-
## 2022 ##
{'Austria': 0.108, 'Belgium': 0.873, 'Bulgaria': 0.299, 'Croatia': 0.476, 'Cyprus': 0.659, 'Czechia': 0.428, 'Denmark': 0.254, 'Estonia': 0.33, 'Finland': 0.592, 'France': 0.63, 'Germany': 0.102, 'Greece': 0.322, 'Hungary': 0.317, 'Ireland': 0.565, 'Italy': 0.727, 'Latvia': 0.223, 'Lithuania': 0.223, 'Luxembourg': 1, 'Malta': 0.031, 'Netherlands': 0.935, 'Poland': 0.259, 'Portugal': 0.204, 'Romania': 0.368, 'Slovakia': 0

In [4]:
dict(zip(years, support_rates))

{2021: 10,
 2022: 11,
 2023: 12,
 2024: 12,
 2025: 13,
 2026: 13,
 2027: 14,
 2028: 14,
 2029: 15,
 2030: 16,
 2031: 16,
 2032: 16,
 2033: 16,
 2034: 16,
 2035: 17,
 2036: 17,
 2037: 18,
 2038: 18,
 2039: 18,
 2040: 18,
 2041: 18,
 2042: 18,
 2043: 18,
 2044: 18,
 2045: 18,
 2046: 18,
 2047: 18,
 2048: 18,
 2049: 19}

In [5]:
scenario_results = []
scenario_countries = []
final_supports = []

for n in range(N_SCENARIOS):
    eu_presidencies = copy(EU_PRESIDENCIES)
    countries_w_bans = copy(COUNTRIES_W_BANS)
    country_rates = COUNTRY_RATES.copy()
    country_growth = COUNTRY_GROWTH.copy()
    report = False
    ban_presidency = False
    proposal = False
    support = len(countries_w_bans)
    done = False
    i = 1
    verbose = (n % 500 == 0)
    if verbose:
        print('## SCENARIO {}/{} ##'.format(n + 1, N_SCENARIOS))

    for year in range(2021, 2050):
        for season in ['H1', 'H2']:
            if not done:
                tick = '{} {}'.format(year, season)
                if verbose:
                    print('{}: {}'.format(i, tick))

                for country in country_rates.keys():
                    # Model cage-free growth in each country per half-year
                    country_rates[country] += (country_growth[country] / 2.0) # TODO: Add error?
                    
                    if country_rates[country] > 1:
                        country_rates[country] = 1

                    # For each country, if 60% or more hens projected to be cage-free in that country,
                    # there is a 46% chance become a NATIONAL restriction in that country
                    if country_rates[country] >= 0.6 and country not in countries_w_bans:
                           if random.random() < 0.46: 
                                if verbose:
                                    print('...{} passed a national restriction!'.format(country))
                                countries_w_bans.append(country)
                                support = len(countries_w_bans)
                                
                if year >= 2022 and verbose:
                    print ("... ...{}/15 countries with restrictions: {}".format(support,
                                                                                 ', '.join(countries_w_bans)))
            
                # Each year, expect a scientific report sometime after December 2022 
                if not report and year >= 2023:
                    if random.random() < 0.25:
                        if verbose:
                            print('...The scientific report arrived!')
                        report = True
                        report_on_tick = i

                # When a report happens, there is a 36% chance of EU PROPOSAL after 2-29 months
                if report and not proposal:
                    if (i == report_on_tick + 1 and random.random() < 0.5) or (i > report_on_tick + 5):
                        if random.random() < 0.36:
                            if verbose:
                                print('...EU propsal made (after scientific report) ')
                            scenario_results.append(year)
                            proposal = True
                 
                # Rotate the EU presidency
                president = eu_presidencies[(i - 1) % len(eu_presidencies)]
                if verbose:
                    print('...{} is President'.format(president))

                # If (a) the EU Presidency is a country with a national law and (b) support is >= 15,
                # and (c) there is a proposal, there is a 57% chance of an EU law after 12-36 months
                if president in countries_w_bans and proposal and support >= 15 and not ban_presidency:
                    if verbose:
                        print('...The EU Presidency has a national ban, support >= 15, and proposal in place... attempting EU ban')
                    ban_presidency = True
                    ban_presidency_on_tick = i
                    ban_country = president

                if ban_presidency:
                    if (i >= ban_presidency_on_tick + 2 and random.random() < 0.5) or (i > ban_presidency_on_tick + 6):
                        if random.random() < 0.57:
                            if verbose:
                                print('...EU BAN ACHIEVED (by {} EU Presidency)! 🎉🎉🎉'.format(ban_country))
                            scenario_results.append(year)
                            scenario_countries.append(ban_country)
                            done = True
                        else:
                            if verbose:
                                print('...attempted EU ban by {} Presidency failed'.format(ban_country))
                            ban_presidency = False

            i += 1
                
    if not done:
        if verbose:
            print('...No EU ban by 2050')
        scenario_results.append(2051)
    
    final_supports.append(support)
        
    if verbose:
        print('-')
    

## SCENARIO 1/10000 ##
1: 2021 H1
...Portugal is President
2: 2021 H2
...Italy passed a national restriction!
...Slovenia passed a national restriction!
...Sweden passed a national restriction!
...Slovenia is President
3: 2022 H1
... ...9/15 countries with restrictions: Germany, Czechia, Luxembourg, France, Netherlands, Austria, Italy, Slovenia, Sweden
...France is President
4: 2022 H2
... ...9/15 countries with restrictions: Germany, Czechia, Luxembourg, France, Netherlands, Austria, Italy, Slovenia, Sweden
...Czechia is President
5: 2023 H1
...Cyprus passed a national restriction!
...Finland passed a national restriction!
... ...11/15 countries with restrictions: Germany, Czechia, Luxembourg, France, Netherlands, Austria, Italy, Slovenia, Sweden, Cyprus, Finland
...Sweden is President
6: 2023 H2
...Belgium passed a national restriction!
... ...12/15 countries with restrictions: Germany, Czechia, Luxembourg, France, Netherlands, Austria, Italy, Slovenia, Sweden, Cyprus, Finland, Belgi

In [6]:
pd.Series(['Not by 2050' if y > 2050 else y for y in scenario_results]).value_counts(normalize=True) * 100

2033           14.155
2030           13.715
2027           10.820
2026            9.800
2028            8.455
2034            6.405
2031            6.360
2029            5.410
2035            4.650
2036            3.765
2024            2.765
2032            2.605
2037            2.380
2023            2.380
2025            1.650
2038            1.570
2040            1.050
2039            0.575
2041            0.465
2042            0.300
2043            0.155
2044            0.155
2045            0.115
2046            0.100
Not by 2050     0.090
2047            0.045
2048            0.035
2049            0.030
dtype: float64

In [7]:
pd.Series(final_supports).value_counts(normalize=True) * 100

16    65.30
17    13.53
18    11.06
15    10.01
19     0.10
dtype: float64

In [8]:
for y in range(2021, 2050):
    print('% chance by {}: {}'.format(y, print_pct(np.sum([s <= y for s in scenario_results]) / len(scenario_results), digits=1)))

% chance by 2021: <0.1%
% chance by 2022: <0.1%
% chance by 2023: 2.4%
% chance by 2024: 5.1%
% chance by 2025: 6.8%
% chance by 2026: 16.6%
% chance by 2027: 27.4%
% chance by 2028: 35.9%
% chance by 2029: 41.3%
% chance by 2030: 55.0%
% chance by 2031: 61.4%
% chance by 2032: 64.0%
% chance by 2033: 78.1%
% chance by 2034: 84.5%
% chance by 2035: 89.2%
% chance by 2036: 92.9%
% chance by 2037: 95.3%
% chance by 2038: 96.9%
% chance by 2039: 97.5%
% chance by 2040: 98.5%
% chance by 2041: 99.0%
% chance by 2042: 99.3%
% chance by 2043: 99.4%
% chance by 2044: 99.6%
% chance by 2045: 99.7%
% chance by 2046: 99.8%
% chance by 2047: 99.8%
% chance by 2048: 99.9%
% chance by 2049: 99.9%


In [8]:
pd.Series(scenario_countries).value_counts(normalize=True) * 100

Austria        35.282339
Luxembourg     19.793753
Netherlands    13.516219
Croatia         7.859431
Slovenia        5.706848
Germany         3.924710
Finland         2.472968
France          2.112535
Denmark         2.052463
Sweden          1.601922
Czechia         1.501802
Spain           1.461754
Belgium         0.951141
Lithuania       0.610733
Italy           0.480577
Cyprus          0.320384
Estonia         0.180216
Ireland         0.170204
dtype: float64